<a href="https://colab.research.google.com/github/jquesada92/delta_lake_project/blob/main/Contraloria.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#📘 Contraloría ETL Pipeline Documentation
<p>This notebook implements a Delta Lake ETL pipeline that processes public employee salary data from the Contraloría General de la República de Panamá. The data is stored in Google Drive as Excel/Parquet files, and the pipeline:

- Reads and stages the data via Spark Structured Streaming
- Applies Slowly Changing Dimension Type 2 (SCD2) logic to track historical changes
- Promotes cleaned and deduplicated records to a Silver layer
- Flags active/inactive employees over time based on presence in source data



## 📁 Folder Structure
/Colab Notebooks/contraloria/

├── staging/         # Incoming files (Parquet)<br>├── checkpoint/      # Checkpoints for streaming jobs<br>
├── spark-warehouse/<br>
│   ├── bronze_scd_type_2/   # Bronze Delta table (SCD2)<br>
│   └── silver/              # Silver Delta table (latest active view)<br>

## ⚙️ Step-by-Step Process
## 📦 Environment Setup
- Install delta-spark dependency


In [9]:
!pip install delta-spark==3.0.0

- Configure paths for staging, checkpoints, and warehouse

In [23]:
# Base path where data is stored in Google Drive
SOURCE_PATH = '/content/drive/MyDrive/Colab Notebooks/contraloria'

# Directory for raw input staging
STAGING_PATH = f'{SOURCE_PATH}/staging'



# Directory used by Spark for checkpointing streaming data
CHECKPOINT_PATH = f'{SOURCE_PATH}/checkpoint'


# Path to Delta Lake Bronze and Silver tables
bronze_table = 'bronze_scd_type_2'
silver_table = 'silver'

# Primary keys used for deduplication and SCD2 logic
key_cols = ['cedula', 'institucion']

# Update timestamp column to track changes
update_col = 'fecha_consulta'

- Start a Spark session with Delta Lake extensions

In [38]:
from pyspark.sql import SparkSession
from delta import *
from delta.tables import DeltaTable
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql import DataFrame
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [12]:

# Create SparkSession with Delta Lake extensions and configurations
builder = SparkSession.builder \
    .appName("DeltaLakeAlternativeSession") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# Finalize Spark session creation
spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [13]:
def create_table_if_not_exists(df: DataFrame, sink_table: str) -> None:

   if not spark.catalog.tableExists(sink_table):
        # Create the table if it does not exist
        df.write.format("delta").mode("overwrite").saveAsTable(sink_table)
        return

In [14]:
# Define a window function used for SCD2 logic
__window = lambda x: Window.partitionBy(*x).orderBy(F.desc(update_col))

##  🧱 Bronze Table: Full Historical Data (SCD Type 2)
Purpose: Capture the full history of changes for each employee (cedula, institución).

How it works:

- Data is ingested from staged Parquet files using Spark Streaming

- Deduplication happens per key + timestamp

- SCD2 fields are added:

  - `start_date`: timestamp of entry

  - `lend_date`: timestamp when record was superseded

  - `last_update`: flag if it's the latest/current version

- Merge logic:

  - Expire old versions (`last_update=False`)

  - Insert new versions (`last_update=True`)

<b>Benefits:</b>

- Full change tracking over time

- Enables time travel queries and audits

In [15]:
def bronze_type2_upsert(microbatch_df: DataFrame, batch_id: str, table_name: str = bronze_table) -> None:
    """
    Perform SCD Type 2 logic to merge a microbatch into the Delta Bronze table.

    Steps:
    - Deduplicates using key columns + update timestamp
    - Adds SCD2 fields: start_date, end_date, is_current
    - Marks outdated records as inactive (last_update=False)
    - Inserts new versioned rows

    Args:
        microbatch_df (DataFrame): Microbatch DataFrame in streaming context
        batch_id (str): ID of the batch (provided by Spark streaming)
        table_name (str): Path to Delta table (default = bronze path)
    """
    if microbatch_df.isEmpty():
        return

    # Deduplicate on key + update timestamp, retain most recent
    df_updates = (
        microbatch_df
        .withColumn("drop_duplicate", F.row_number().over(__window(key_cols + [update_col])))
        .filter(F.col("drop_duplicate") == 1)
        .drop("drop_duplicate")

        # Apply SCD2 fields
        .withColumn("row_num", F.row_number().over(__window(key_cols)))
        .withColumn("start_date", F.col(update_col))
        .withColumn("end_date", F.lag(update_col).over(__window(key_cols)))
        .withColumn("last_update", F.when(F.col("row_num") == 1, True).otherwise(False))
        .drop("row_num")
    )

    # Ensure the table exists
    create_table_if_not_exists(df_updates, table_name)

    # Get reference to existing Delta table
    delta_target = DeltaTable.forName(spark, table_name)
    updates = df_updates.alias("updates")
    target = delta_target.alias("target")

    # Build merge condition based on keys
    merge_condition = " AND ".join([f"bronze_target.{k} = bronze_updates.{k}" for k in key_cols])

    # Merge logic:
    # 1. When matched with new data, update old record to inactive (last_update = false)
    # 2. Insert new record with updated data
    delta_target.alias("bronze_target").merge(
        updates.alias("bronze_updates"),
        f"({merge_condition} AND bronze_target.last_update = true) AND "
        f"(bronze_updates.last_update = true) AND "
        f"(bronze_updates.{update_col} > bronze_target.{update_col})"
    ).whenMatchedUpdate(
        set={"end_date": "bronze_updates.start_date", "last_update": F.lit(False)}
    ).whenNotMatchedInsertAll().execute()

In [16]:
# Define schema to match expected structure of Excel files converted to Parquet
schema = StructType([
    StructField('nombre', StringType(), True),
    StructField('apellido', StringType(), True),
    StructField('cedula', StringType(), True),
    StructField('cargo', StringType(), True),
    StructField('salario', DoubleType(), True),
    StructField('gasto', DoubleType(), True),
    StructField('estado', StringType(), True),
    StructField('fecha_de_inicio', DateType(), True),
    StructField('fecha_actualizacion', TimestampType(), True),
    StructField('fecha_consulta', TimestampType(), True),
    StructField('archivo', StringType(), True),
    StructField('institucion', StringType(), True)
])

# Create streaming DataFrame from Parquet files written by ingestion pipeline
source_staging_sdf = (
    spark.readStream.format("parquet")
    .schema(schema)
    .parquet(STAGING_PATH)
    .withColumn('file_path', F.input_file_name())
)


# Bronze ingestion using foreachBatch and SCD2 upsert logic
bronze_query = (
    source_staging_sdf.writeStream
    .trigger(availableNow=True)  # Process files immediately
    .foreachBatch(lambda df, batch_id: bronze_type2_upsert(df, batch_id))
    .option("checkpointLocation", CHECKPOINT_PATH + '/bronze')
    .outputMode("append")
    .start()
)

# Wait for streaming query to finish
bronze_query.awaitTermination()

| Campo / Field         | Descripción (ES)                                                               | Description (EN)                                                |
| --------------------- | ------------------------------------------------------------------------------ | --------------------------------------------------------------- |
| `nombre`              | Nombre de la persona                                                           | First name of the person                                        |
| `apellido`            | Apellido de la persona                                                         | Last name of the person                                         |
| `cedula`              | Identificación personal única                                                  | Unique personal identification number                           |
| `cargo`               | Cargo o puesto que ocupa la persona                                            | Job title or position held                                      |
| `salario`             | Salario base recibido                                                          | Base salary received                                            |
| `gasto`               | Gastos de representación asociados al cargo                                    | Representation or administrative expenses                       |
| `estado`              | Estado del contrato: permanente o temporal                                     | Contract status: permanent or temporary                         |
| `fecha_de_inicio`     | Fecha en la que la persona inició funciones en la institución                  | Date when the person began duties in the institution            |
| `fecha_actualizacion` | Fecha en la que el portal web actualizó por última vez su información          | Date when the website last updated its data                     |
| `fecha_consulta`      | Fecha en la que el script de web scraping extrajo la información               | Date when the web scraping script retrieved the information     |
| `archivo`             | Nombre del archivo descargado desde la web                                     | Name of the downloaded file                                     |
| `institucion`         | Institución o entidad gubernamental en la que trabaja la persona               | Government institution or ministry where the person works       |
| `file_path`           | Ruta donde se almacenó el archivo descargado                                   | Path where the downloaded file was saved                        |
| `start_date`          | Fecha desde la cual el registro se considera vigente (inicio del periodo SCD2) | Start date of the record's validity (SCD Type 2 period start)\_ |


### Bronze Analysis

In [17]:
# Load the Bronze Delta table and preprocess the data
bronze_sdf = (
    spark.read.table(bronze_table)

        # Remove duplicate records based on key identifying columns and update timestamp
        # This ensures historical data is not artificially inflated by repeated records
        .drop_duplicates(subset=[
            'cedula', 'nombre', 'apellido', 'estado', 'institucion', 'cargo',
            'fecha_de_inicio', 'salario', 'gasto',
            'fecha_actualizacion'
        ])
        .withColumn("total_salary", F.col('salario') + F.col('gasto'))
        .withColumn(
            "delta_total_salary",
            F.col('total_salary')
            - F.lag('total_salary', -1)
              .over(Window.partitionBy('cedula').orderBy(F.col(update_col).desc()))
        )
)

# Aggregate and analyze the most frequently updated records by 'cedula'
agg_updates_by_cedula_bronze_df = (
    bronze_sdf
        # Group records by 'cedula'
        .groupBy('cedula')

        # Count the number of distinct update timestamps per person
        .agg(F.countDistinct('fecha_actualizacion').alias('updates_count'))

        # Sort in descending order to get the most updated records
        .orderBy(F.col('updates_count').desc())

        # Limit to the top 5 most frequently updated 'cedulas'
        .limit(5)

        # Convert the result to a Pandas DataFrame for further analysis or visualization
        .toPandas()
)

### Top Cédula with the Highest Number of Updates

In [18]:
agg_updates_by_cedula_bronze_df.set_index('cedula')

,updates_count
cedula,
8-0886-001530,7
8-0804-001227,7
1-0738-000761,7
4-0763-000747,7
5-0710-002090,7


In [19]:
# The following record corresponds to the individual whose cédula (ID number) appears most frequently in our dataset, indicating the highest number of updates or changes over time:
example = 'cedula="8-0886-001530"'

In [40]:
bronze_sdf.where(example)\
.select(['cedula','nombre','apellido','estado','institucion','cargo','fecha_de_inicio','salario','gasto',"total_salary","delta_total_salary",'fecha_actualizacion','fecha_consulta','start_date','end_date','last_update'])\
.distinct().orderBy(F.col(update_col).desc()).toPandas()

,cedula,nombre,apellido,estado,institucion,cargo,fecha_de_inicio,salario,gasto,total_salary,delta_total_salary,fecha_actualizacion,fecha_consulta,start_date,end_date,last_update
0,8-0886-001530,ALICIA,REINA,PERMANENTE,TRIBUNAL ELECTORAL,SECRETARIA II,2018-07-16,1000.00,0.00,1000.00,0.00,2025-07-16 08:20:06,2025-07-19 20:37:30.979124,2025-07-19 20:37:30.979124,NaT,True
1,8-0886-001530,ALICIA,REINA,PERMANENTE,TRIBUNAL ELECTORAL,SECRETARIA II,2018-07-16,1000.00,0.00,1000.00,-100.00,2025-02-03 08:20:06,2025-02-06 17:54:05.017198,2025-02-06 17:54:05.017198,2025-07-19 15:59:40.777187,False
2,8-0886-001530,ALICIA,REINA,EVENTUAL,TRIBUNAL ELECTORAL,ASISTENTE ADMINISTRATIVO I,2024-02-01,1100.00,0.00,1100.00,0.00,2024-05-03 08:20:07,2024-05-13 20:57:52.517410,2024-05-13 20:57:52.517410,2025-02-06 17:54:05.017198,False
3,8-0886-001530,ALICIA,REINA,EVENTUAL,TRIBUNAL ELECTORAL,ASISTENTE ADMINISTRATIVO I,2024-01-02,1100.00,0.00,1100.00,200.00,2024-04-02 08:20:09,2024-04-09 19:38:40.000000,2024-04-09 19:38:40.000000,2024-05-13 20:57:52.517410,False
4,8-0886-001530,ALICIA,REINA,PERMANENTE,TRIBUNAL ELECTORAL,SECRETARIA II,2018-07-16,900.00,0.00,900.00,75.00,2024-03-18 08:20:07,2024-04-01 11:36:27.000000,2024-04-01 11:36:27.000000,2024-04-09 19:38:40.000000,False
5,8-0886-001530,ALICIA,REINA,PERMANENTE,TRIBUNAL ELECTORAL,SECRETARIA II,2018-07-16,825.00,0.00,825.00,0.00,2024-03-02 08:20:08,2024-03-05 18:54:20.000000,2024-03-05 18:54:20.000000,2024-04-01 11:36:27.000000,False
6,8-0886-001530,ALICIA,REINA,EVENTUAL,TRIBUNAL ELECTORAL,SECRETARIA II,2018-07-16,825.00,0.00,825.00,NaN,2024-02-19 08:20:07,2024-02-29 20:55:34.000000,2024-02-29 20:55:34.000000,2024-03-05 18:54:20.000000,False


Observations for cédula "8-0886-001530":
- Her most recent salary is not the highest recorded in her history.

- She experienced a job change along with a salary increase from **`$825` to `$1100`**.

- Later, she returned to her previous role with a salary decrease of **`$100`**, and is currently earning **$ 1000**.

## 🥈 Silver Table: Latest Active Snapshot (SCD Type 1 + Active Flag)
Purpose: Maintain a simplified view of the latest, active employees for analytics.

<b>How it works:</b>

- Loads only current records (last_update = true) from Bronze

- Deduplicates to retain most recent per employee + institution

- Adds active=True flag

- Updates existing records in Silver or inserts new ones

- Marks records as inactive if they're not in the latest batch

<b>Benefits:</b>

- Fast querying and dashboarding

- Tracks whether an employee is currently listed in public records

In [24]:
def silver_upsert(table_name: str = silver_table) -> None:
    """
    Performs an upsert into the Silver Delta table based on the latest Bronze records.
    Steps:
    - Loads only the 'current' (last_update=True) records from Bronze
    - Deduplicates again to avoid redundancy
    - Keeps the latest update per institution
    - Sets active=True for current employees
    - Marks records as inactive when missing in the latest batch

    Args:
        table_name (str): Path to Silver Delta table
    """
    # Load active records from Bronze layer
    __bronze_sdf = (
        spark.read.format('delta').table(bronze_table)
        .where("last_update = true")
        .withColumn(
            "drop_duplicate",
            F.row_number().over(__window(key_cols))
        )
        .where('drop_duplicate=1')  # Keep the most recent per key+timestamp
        .select([
            'nombre', 'apellido', 'cedula', 'cargo', 'salario', 'gasto', 'estado',
            'fecha_de_inicio', 'institucion', update_col
        ])
        .withColumn('active', F.lit(True))  # Add active flag
        .distinct()
    )

    # Ensure Silver table exists
    create_table_if_not_exists(__bronze_sdf, silver_table)
    # Get the latest update per institution (to determine activity)
    __df_updates = (
        __bronze_sdf
        .withColumn('last_update', F.max(update_col).over(__window(['institucion'])))
        .where(F.col('last_update') == F.col(update_col))
        .drop('last_update')
        .distinct()
    )

    # Get reference to Silver Delta table
    __delta_target = DeltaTable.forName(spark, table_name)
    __updates = __df_updates.alias("silver_updates")
    __target = __delta_target.alias("silver_target")

    # Build merge condition on primary keys, explicitly referencing aliases
    merge_condition = " AND ".join([f"(silver_target.{k} = silver_updates.{k})" for k in key_cols])
    print(merge_condition)
    # Merge logic:
    # - When matched, update all fields
    # - When record is missing in updates, mark as inactive
    __target.merge(
        __updates,
        merge_condition
    )\
     .whenNotMatchedBySourceUpdate(condition='silver_target.active = true',set={'silver_target.active': F.lit(False)})\
     .whenMatchedUpdateAll()\
     .whenNotMatchedInsertAll(condition= 'silver_updates.active = true')\
     .execute()

In [25]:
# Execute silver layer update process
silver_upsert(silver_table)

(silver_target.cedula = silver_updates.cedula) AND (silver_target.institucion = silver_updates.institucion)


In [35]:
silver_sdf = spark.read.table(silver_table)
summary_status_silver_df = silver_sdf.groupBy(['institucion']).pivot("active").count().withColumnsRenamed({'false':'Inactive','true':'Active'}).withColumn('total',F.col("Inactive") + F.col('Active')).join(
    silver_sdf.where('active=true').groupBy('institucion').agg(F.sum('salario').alias('sum_of_salaries'), F.sum('gasto').alias('sum_of_representation_allowance')).withColumn('total_cost',F.col('sum_of_salaries') + F.col('sum_of_representation_allowance')),on='institucion')


In [39]:

summary_status_silver_df.toPandas().sort_values('total_cost',ascending=False)

,institucion,Inactive,Active,total,sum_of_salaries,sum_of_representation_allowance,total_cost
9,MINISTERIO DE EDUCACION,13829,59866,73695,101196533.49,17000.00,101213533.49
7,MINISTERIO DE SALUD,3242,19607,22849,41514382.28,40100.00,41554482.28
11,MINISTERIO DE SEGURIDAD PUBLICA,3426,30000,33426,40437748.34,484100.00,40921848.34
27,ORGANO JUDICIAL,793,6868,7661,13039840.81,1414250.00,14454090.81
1,PROCURADURÍA GENERAL DE LA NACIÓN,661,7631,8292,12994490.76,722250.00,13716740.76
18,CONTRALORIA GENERAL DE LA REPUBLICA,627,4674,5301,7421840.50,42338.00,7464178.50
20,MINISTERIO DE LA PRESIDENCIA,2665,4699,7364,6545709.94,73250.00,6618959.94
3,ASAMBLEA NACIONAL,2476,3042,5518,5758643.00,242500.00,6001143.00
15,MINISTERIO DE DESARROLLO AGROPECUARIO,791,2440,3231,4321097.72,11100.00,4332197.72
25,TRIBUNAL ELECTORAL,488,3282,3770,3997311.00,22300.00,4019611.00


# 🔍 Summary
| Layer  | Format | Type                       | Purpose                                  |
| ------ | ------ | -------------------------- | ---------------------------------------- |
| Bronze | Delta  | SCD Type 2 (historical)    | Track changes to each employee over time |
| Silver | Delta  | SCD Type 1 + `active` flag | Get latest snapshot with activity status |


## 💡 Why SCD2?
Without SCD2, we’d overwrite the old data and lose historical visibility. This makes:

- Audits impossible

- Delayed records inaccurate

- Snapshots heavier and harder to manage

With SCD2 in Bronze + active tracking in Silver:

- You get complete lineage

- You can build daily views without duplicating storage

- You reduce processing cost while preserving fidelity